# fetching gnd feature code descriptions

In [ ]:
from acdh_tei_pyutils.tei import TeiReader
from acdh_tei_pyutils.utils import extract_fulltext
from tqdm import tqdm

In [ ]:
nsmap = {
    "gn": "https://www.geonames.org/ontology#",
    "skos": "http://www.w3.org/2004/02/skos/core#",
    "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#"
}

In [ ]:
doc = TeiReader("https://www.geonames.org/ontology/ontology_v3.3.rdf")

In [ ]:
place_types = PlaceType.objects.filter(name__icontains="http")
for x in tqdm(place_types):
    name = x.name.split('#')[-1]
    x.name = name
    x.save()    

In [ ]:
place_types = PlaceType.objects.exclude(name__icontains="(").filter(name__icontains=".")

In [ ]:
for x in tqdm(place_types):
    xpath_expr = f'.//gn:Code[@rdf:about="#{x.name}"]/skos:definition[@xml:lang="en"]'
    try:
        description = doc.tree.xpath(xpath_expr, namespaces=nsmap)[0]
    except IndexError:
        continue
    x.description = f"{extract_fulltext(description).title()}. Quelle: https://www.geonames.org/ontology/ontology_v3.3.rdf"
    x.save()